In [10]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [11]:
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [12]:
data.shape

(15999, 2)

In [13]:
data.isna().sum()

Text        0
Emotions    0
dtype: int64

In [40]:
data['Emotions'].value_counts()

joy         5362
sadness     4665
anger       2159
fear        1937
love        1304
surprise     572
Name: Emotions, dtype: int64

- As this is a problem of natural language processing, I’ll start by tokenizing the data

In [14]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

- Now we need to pad the sequences to the same length to feed them into a neural network. Here’s how we can pad the sequences of the texts to have the same length

In [15]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

- Now I’ll use the label encoder method to convert the classes from strings to a numerical representation

In [16]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

- We are now going to One-hot encode the labels. One hot encoding refers to the transformation of categorical labels into a binary representation where each label is represented as a vector of all zeros except a single 1. This is necessary because machine learning algorithms work with numerical data. So here is how we can One-hot encode the labels

In [17]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

# Text Emotions Classification Model
- Now we will split the data into training and test sets:

In [18]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

- Now let’s define a neural network architecture for our classification problem and use it to train a model to classify emotions

In [19]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 28s 50ms/step - loss: 1.3654 - accuracy: 0.4790 - val_loss: 0.8709 - val_accuracy: 0.7325
Epoch 2/10
400/400 [==============================] - 19s 48ms/step - loss: 0.3525 - accuracy: 0.8934 - val_loss: 0.5057 - val_accuracy: 0.8331
Epoch 3/10
400/400 [==============================] - 19s 48ms/step - loss: 0.0557 - accuracy: 0.9861 - val_loss: 0.5276 - val_accuracy: 0.8347
Epoch 4/10
400/400 [==============================] - 19s 48ms/step - loss: 0.0242 - accuracy: 0.9952 - val_loss: 0.6020 - val_accuracy: 0.8272
Epoch 5/10
400/400 [==============================] - 19s 48ms/step - loss: 0.0158 - accuracy: 0.9967 - val_loss: 0.5868 - val_accuracy: 0.8281
Epoch 6/10
400/400 [==============================] - 19s 48ms/step - loss: 0.0156 - accuracy: 0.9972 - val_loss: 0.6221 - val_accuracy: 0.8331
Epoch 7/10
400/400 [==============================] - 19s 48ms/step - loss: 0.0134 - accuracy: 0.9970 - val_loss: 0.6279 - val_accuracy:

In [38]:
input_text = "i am feeling like jumping and dancing after getting the job"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)a

1/1 [==============================] - 0s 56ms/step
['joy']
